In [1]:
pip install mesa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Imports
import random
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.time import RandomActivation

In [3]:
# Definir la clase del agente robot
class RobotAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.has_box = False
        self.box_stack = 0
        self.steps_taken = 0

    # Movimiento del agente robot mediante las celdas vecinas de Von Newmann
    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos,
            moore=False,
            include_center=False
        )
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)
        self.steps_taken += 1

    # Recoger y buscar cajas en celdas vecinas
    def pickup_box(self):
        if self.box_stack < self.model.max_stack_height:
            neighbors = self.model.grid.get_neighborhood(
                self.pos,
                moore=False,
                include_center=False
            )
            for neighbor_pos in neighbors:
                cellmates = self.model.grid.get_cell_list_contents([neighbor_pos])
                for agent in cellmates:
                    if isinstance(agent, BoxAgent):
                        self.has_box = True
                        self.model.grid.remove_agent(agent)
                        self.model.schedule.remove(agent)
                        self.box_stack += 1
                        return

    # Dejar cajas mediante la verificación de número máximo de cajas cargadas
    def drop_box(self):
        if self.has_box:
            self.has_box = False
            new_box = BoxAgent(self.model.num_boxes + len(self.model.schedule.agents), self.model)
            self.box_stack -= 1

            if self.box_stack == 0 and self.model.stacks_created < self.model.num_stacks:
                empty_coords = self.model.get_empty_coords()
                if empty_coords:
                    x, y = self.random.choice(empty_coords)
                    self.model.grid.place_agent(new_box, (x, y))
                else:
                    self.model.grid.place_agent(new_box, self.pos)
                self.model.place_box()
            else:
                empty_coords = self.model.get_empty_coords()
                if empty_coords:
                    x, y = self.random.choice(empty_coords)
                    self.model.grid.place_agent(new_box, (x, y))
                else:
                    self.model.grid.place_agent(new_box, self.pos)
            self.model.schedule.add(new_box)

    # Aletoriedad en el movimiento de los agentes
    def step(self):
        if random.random() < 0.5:
            self.move()
            if not self.has_box:
                self.pickup_box()
            else:
                self.drop_box()

In [4]:
# Definir la clase del agente caja
class BoxAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)

In [20]:
# Definir la clase del modelo
class WarehouseModel(Model):
    def __init__(self, width, height, num_boxes, num_robots, max_stack_height, max_time_steps):
        super().__init__()
        self.width = width
        self.height = height
        self.num_boxes = num_boxes
        self.num_robots = num_robots
        self.max_stack_height = max_stack_height
        self.max_time_steps = max_time_steps
        self.num_stacks = num_boxes // max_stack_height
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)
        self.running = True
        self.total_moves = 0
        self.stacks_created = 0
        self.steps_count = 0

        # Agregar robots
        for i in range(self.num_robots):
            robot = RobotAgent(i, self)
            self.schedule.add(robot)
            empty_coords = self.get_empty_coords()
            x, y = self.random.choice(empty_coords)
            self.grid.place_agent(robot, (x, y))

        # Agregar cajas
        for i in range(self.num_boxes):
            box = BoxAgent(i, self)
            self.schedule.add(box)
            empty_coords = self.get_empty_coords()
            x, y = self.random.choice(empty_coords)
            self.grid.place_agent(box, (x, y))

    # Obtener coordenadas vacías
    def get_empty_coords(self):
        empty_coords = [(x, y) for x in range(self.width) for y in range(self.height) if self.grid.is_cell_empty((x, y))]
        return empty_coords

    # Número de pilas creadas
    def place_box(self):
        self.stacks_created += 1

    # Contador de movimientos
    def step(self):
        moves_this_step = 0
        self.schedule.step()
        moves_this_step += sum(agent.steps_taken for agent in self.schedule.agents if isinstance(agent, RobotAgent))
        self.total_moves += moves_this_step

        # Verificar si se han creado todos los montones o si se ha alcanzado el límite de pasos
        if self.stacks_created >= self.num_stacks or self.total_moves >= self.max_time_steps:
            self.running = False

In [21]:
# Parámetros de la simulación
width = 10
height = 10
num_boxes = 20
num_robots = 5
max_stack_height = 5
max_time_steps = 100

In [22]:
# Ejecutar la simulación
model = WarehouseModel(width, height, num_boxes, num_robots, max_stack_height, max_time_steps)

while model.running:
    model.step()

# Imprimir los resultados
print("Número total de movimientos realizados por todos los robots:", model.total_moves)
print("Número de montones creados:", model.stacks_created)

Número total de movimientos realizados por todos los robots: 117
Número de montones creados: 4
Número total de pasos de la simulación: 9
